# Building a call for the Census Bureau API:

This tutorial will start with a very basic call using the python requests package and add complexity as it goes along.

**Resources**
1. Census Bureau API call examples: https://www.census.gov/data/developers/guidance/api-user-guide/example-api-queries.html  
2. Info on requests package: https://requests.readthedocs.io/en/master/user/quickstart/  
3. Info on json package: https://docs.python.org/3/library/json.html  
4. List of ACS 5-Year Estimate variables: https://api.census.gov/data/2020/acs/acs5/variables.html  

## Import packages

In [1]:
import requests #interface with the API
import json #formatting json files
import pandas as pd #manipulating dataframe
import pickle #save your API key

## Create a pickle file with your API key so that it is not public in your notebook

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [3]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

## Basic call using the **requests** package  
This example puts a url string that follows the Census Bureau's example straight into requests.get() and names this response "data".

In [4]:
#url string is what you pass into the requests.get() function. It includes the year, series, variables, and geographic information
url_str = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,GEO_ID,B01003_001E&for=state:*'
data = requests.get(url_str)

Check "data" out to see what it is we're getting,

In [5]:
data

<Response [200]>

The Census API returns a 'response object' in json format (you can see from the output above that it is a response object with a success code 200, I just know that it is a json object because that is a very common format - aka a javascript object), when you use the data.json() command (demonstrated below) you can see the json formatted object.

In [6]:
data.json()

[['NAME', 'GEO_ID', 'B01003_001E', 'state'],
 ['Alabama', '0400000US01', '4876250', '01'],
 ['Alaska', '0400000US02', '737068', '02'],
 ['Arizona', '0400000US04', '7050299', '04'],
 ['Arkansas', '0400000US05', '2999370', '05'],
 ['California', '0400000US06', '39283497', '06'],
 ['Colorado', '0400000US08', '5610349', '08'],
 ['Delaware', '0400000US10', '957248', '10'],
 ['District of Columbia', '0400000US11', '692683', '11'],
 ['Connecticut', '0400000US09', '3575074', '09'],
 ['Florida', '0400000US12', '20901636', '12'],
 ['Georgia', '0400000US13', '10403847', '13'],
 ['Idaho', '0400000US16', '1717750', '16'],
 ['Hawaii', '0400000US15', '1422094', '15'],
 ['Illinois', '0400000US17', '12770631', '17'],
 ['Indiana', '0400000US18', '6665703', '18'],
 ['Iowa', '0400000US19', '3139508', '19'],
 ['Kansas', '0400000US20', '2910652', '20'],
 ['Kentucky', '0400000US21', '4449052', '21'],
 ['Louisiana', '0400000US22', '4664362', '22'],
 ['Maine', '0400000US23', '1335492', '23'],
 ['Maryland', '04

We want to make a pandas dataframe out of this json object. You can write that in using the code below.

In [7]:
url_str = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,GEO_ID,B01003_001E&for=state:*'
data = requests.get(url_str)
data = pd.DataFrame(data=data.json())

In [8]:
data.head()

,0,1,2,3
0,NAME,GEO_ID,B01003_001E,state
1,Alabama,0400000US01,4876250,01
2,Alaska,0400000US02,737068,02
3,Arizona,0400000US04,7050299,04
4,Arkansas,0400000US05,2999370,05


This gives us a clean indexed dataframe with default column headers. Say we want the first row of data to be the column header though.... you can index into the correct parts of the data by designating the 'data' and 'columns' in the pd.DataFrame() command.

In [9]:
url_str = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,GEO_ID,B01003_001E&for=state:*'
data = requests.get(url_str)
data = pd.DataFrame(data=data.json()[1:], columns = data.json()[0:1])

In [10]:
data.head()

,NAME,GEO_ID,B01003_001E,state
0,Alabama,0400000US01,4876250,01
1,Alaska,0400000US02,737068,02
2,Arizona,0400000US04,7050299,04
3,Arkansas,0400000US05,2999370,05
4,California,0400000US06,39283497,06


However... we don't necessarily know what all of these variables are. We can designate a **list** of column names if we choose.

In [11]:
url_str = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,GEO_ID,B01003_001E&for=state:*'
data = requests.get(url_str)
col_names = ['State','GEOID','Total Population','StateFIPS']
data = pd.DataFrame(data=data.json()[1:], columns = col_names)

In [12]:
data.head()

,State,GEOID,Total Population,StateFIPS
0,Alabama,0400000US01,4876250,01
1,Alaska,0400000US02,737068,02
2,Arizona,0400000US04,7050299,04
3,Arkansas,0400000US05,2999370,05
4,California,0400000US06,39283497,06


Now we can use this to call multiple variables that we may need to make a calculation, say the population by race series.

In [13]:
url_str = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,GEO_ID,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E&for=state:*'
data = requests.get(url_str)
col_names = ['State','GEOID','Total Population','White Alone','Black or African American Alone','American Indian and Alaska Native Alone','Asian Alone',
             'Native Hawaiian and Other Pacific Islander Alone','Some Other Race Alone','Two or More Races','StateFIPS']
data = pd.DataFrame(data=data.json()[1:], columns = col_names)

In [14]:
data.head()

,State,GEOID,Total Population,White Alone,Black or African American Alone,American Indian and Alaska Native Alone,Asian Alone,Native Hawaiian and Other Pacific Islander Alone,Some Other Race Alone,Two or More Races,StateFIPS
0,Alabama,0400000US01,4876250,3320247,1299048,25565,66270,2238,70662,92220,01
1,Alaska,0400000US02,737068,476015,24205,109751,45920,9204,11308,60665,02
2,Arizona,0400000US04,7050299,5444453,317462,317414,233213,14458,460262,263037,04
3,Arkansas,0400000US05,2999370,2301044,459542,20434,45504,8733,83576,80537,05
4,California,0400000US06,39283497,23453222,2274108,303998,5692423,155290,5481792,1922664,06


This starts to get tedious as the number of variables exceeds our screen limit. We can use a dictionary called 'predicates' to populate our "get" and our "for" portions of the API call so that we're dealing with lists and not a super long tedious html string.

In [15]:
url_str= 'https://api.census.gov/data/2019/acs/acs5?key='+api_key
predicates= {}
get_vars= ["NAME", "GEO_ID",'B02001_001E','B02001_002E','B02001_003E','B02001_004E','B02001_005E','B02001_006E','B02001_007E','B02001_008E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:*"
data= requests.get(url_str, params= predicates)
col_names = ['State','GEOID','Total Population','White Alone','Black or African American Alone','American Indian and Alaska Native Alone','Asian Alone',
             'Native Hawaiian and Other Pacific Islander Alone','Some Other Race Alone','Two or More Races','StateFIPS']
data=pd.DataFrame(columns=col_names, data=data.json()[1:])
#let's also add a print line here to let us know when the process is complete
print('Your API call is complete')

Your API call is complete


In [16]:
data.head()

,State,GEOID,Total Population,White Alone,Black or African American Alone,American Indian and Alaska Native Alone,Asian Alone,Native Hawaiian and Other Pacific Islander Alone,Some Other Race Alone,Two or More Races,StateFIPS
0,Alabama,0400000US01,4876250,3320247,1299048,25565,66270,2238,70662,92220,01
1,Alaska,0400000US02,737068,476015,24205,109751,45920,9204,11308,60665,02
2,Arizona,0400000US04,7050299,5444453,317462,317414,233213,14458,460262,263037,04
3,Arkansas,0400000US05,2999370,2301044,459542,20434,45504,8733,83576,80537,05
4,California,0400000US06,39283497,23453222,2274108,303998,5692423,155290,5481792,1922664,06


Check to see what 'predicates' looks like...

In [17]:
predicates

{'get': 'NAME,GEO_ID,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E',
 'for': 'state:*'}

Say we wanted to get all counties in Tennessee...

In [18]:
url_str= 'https://api.census.gov/data/2019/acs/acs5?key='+api_key
predicates= {}
get_vars= ["NAME", "GEO_ID",'B02001_001E','B02001_002E','B02001_003E','B02001_004E','B02001_005E','B02001_006E','B02001_007E','B02001_008E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates['in']= 'state:47'
data= requests.get(url_str, params= predicates)
col_names = ['State','GEOID','Total Population','White Alone','Black or African American Alone','American Indian and Alaska Native Alone','Asian Alone',
             'Native Hawaiian and Other Pacific Islander Alone','Some Other Race Alone','Two or More Races','StateFIPS','CountyFIPS']
data=pd.DataFrame(columns=col_names, data=data.json()[1:])
#let's also add a print line here to let us know when the process is complete
print('Your API call is complete')

Your API call is complete


In [19]:
data.head()

,State,GEOID,Total Population,White Alone,Black or African American Alone,American Indian and Alaska Native Alone,Asian Alone,Native Hawaiian and Other Pacific Islander Alone,Some Other Race Alone,Two or More Races,StateFIPS,CountyFIPS
0,"Sumner County, Tennessee",0500000US47165,183437,159883,13457,465,2564,164,2818,4086,47,165
1,"Trousdale County, Tennessee",0500000US47169,10231,8690,1042,54,25,0,30,390,47,169
2,"Clay County, Tennessee",0500000US47027,7654,7258,183,0,8,0,137,68,47,027
3,"Shelby County, Tennessee",0500000US47157,936374,366180,502669,1562,24626,264,25456,15617,47,157
4,"Henderson County, Tennessee",0500000US47077,27977,25091,2192,0,35,22,31,606,47,077


Say you want to compare counties in a few states. You could call all counties in all states and filter your results based on the StateFIPS field, or you could loop through a list of states. First, we'll loop.

In [24]:
states = ['17', '47']

In [25]:
#use a for loop to do two states
data_appended = [] #intialize list
for i in states:
    url_str= 'https://api.census.gov/data/2019/acs/acs5?key='+api_key
    predicates= {}
    get_vars= ["NAME", "GEO_ID", 'B01001_001E','B01001_002E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "county:*"
    predicates["in"]= "state:{}".format(i)
    data= requests.get(url_str, params= predicates)
    col_names = ['County', 'GEOID','total','mtotal','StateFIPS', 'CountyFIPS']
    data=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype = str)
    data_appended.append(data) #append the current instance to the last
data_appended = pd.concat(data_appended) 
data = data_appended
print('Your API call is complete')

Your API call is complete


In [27]:
data.head()

,County,GEOID,total,mtotal,StateFIPS,CountyFIPS
0,"Fayette County, Illinois",0500000US17051,21565,11469,17,051
1,"Logan County, Illinois",0500000US17107,29003,14298,17,107
2,"Saline County, Illinois",0500000US17165,23994,11807,17,165
3,"Lake County, Illinois",0500000US17097,701473,350466,17,097
4,"Massac County, Illinois",0500000US17127,14219,6714,17,127


In [28]:
data.tail()

,County,GEOID,total,mtotal,StateFIPS,CountyFIPS
90,"Monroe County, Tennessee",0500000US47123,46064,22679,47,123
91,"Henry County, Tennessee",0500000US47079,32284,15498,47,079
92,"Crockett County, Tennessee",0500000US47033,14399,6834,47,033
93,"Lake County, Tennessee",0500000US47095,7401,4641,47,095
94,"Knox County, Tennessee",0500000US47093,461104,224184,47,093


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197 entries, 0 to 94
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   County      197 non-null    object
 1   GEOID       197 non-null    object
 2   total       197 non-null    object
 3   mtotal      197 non-null    object
 4   StateFIPS   197 non-null    object
 5   CountyFIPS  197 non-null    object
dtypes: object(6)
memory usage: 10.8+ KB


You are also able to call all data and filter out other states.

In [32]:
url_str= 'https://api.census.gov/data/2019/acs/acs5?key='+api_key
predicates= {}
get_vars= ["NAME", "GEO_ID",'B02001_001E','B02001_002E','B02001_003E','B02001_004E','B02001_005E','B02001_006E','B02001_007E','B02001_008E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates['in']= 'state:*'
data= requests.get(url_str, params= predicates)
col_names = ['State','GEOID','Total Population','White Alone','Black or African American Alone','American Indian and Alaska Native Alone','Asian Alone',
             'Native Hawaiian and Other Pacific Islander Alone','Some Other Race Alone','Two or More Races','StateFIPS','CountyFIPS']
data=pd.DataFrame(columns=col_names, data=data.json()[1:])
#let's also add a print line here to let us know when the process is complete
print('Your API call is complete')

Your API call is complete


In [36]:
test = data.loc[(data['StateFIPS'] == '01') | (data['StateFIPS'] == '47')]

In [37]:
test.head()

,State,GEOID,Total Population,White Alone,Black or African American Alone,American Indian and Alaska Native Alone,Asian Alone,Native Hawaiian and Other Pacific Islander Alone,Some Other Race Alone,Two or More Races,StateFIPS,CountyFIPS
286,"Sumner County, Tennessee",0500000US47165,183437,159883,13457,465,2564,164,2818,4086,47,165
287,"Trousdale County, Tennessee",0500000US47169,10231,8690,1042,54,25,0,30,390,47,169
288,"Clay County, Tennessee",0500000US47027,7654,7258,183,0,8,0,137,68,47,027
408,"Shelby County, Tennessee",0500000US47157,936374,366180,502669,1562,24626,264,25456,15617,47,157
409,"Henderson County, Tennessee",0500000US47077,27977,25091,2192,0,35,22,31,606,47,077


In [38]:
test.tail()

,State,GEOID,Total Population,White Alone,Black or African American Alone,American Indian and Alaska Native Alone,Asian Alone,Native Hawaiian and Other Pacific Islander Alone,Some Other Race Alone,Two or More Races,StateFIPS,CountyFIPS
3213,"Monroe County, Tennessee",0500000US47123,46064,43415,961,295,185,0,416,792,47,123
3214,"Henry County, Tennessee",0500000US47079,32284,28754,2643,84,127,0,165,511,47,079
3215,"Crockett County, Tennessee",0500000US47033,14399,11447,1877,17,48,6,679,325,47,033
3216,"Lake County, Tennessee",0500000US47095,7401,4997,2110,10,18,0,112,154,47,095
3217,"Knox County, Tennessee",0500000US47093,461104,394105,40147,1306,10114,205,4890,10337,47,093


We can change our geographies easily, but some combinations require a crosswalk - for example you'd need to get all places in Tennessee and then find a crosswalk for counties because places in a county is an "unsupported hierarchy".

In [12]:
url_str= 'https://api.census.gov/data/2019/acs/acs5?key='+api_key
predicates= {}
get_vars= ["NAME", "GEO_ID",'B02001_001E','B02001_002E','B02001_003E','B02001_004E','B02001_005E','B02001_006E','B02001_007E','B02001_008E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates['in']= 'state:47'
data= requests.get(url_str, params= predicates)
col_names = ['State','GEOID','Total Population','White Alone','Black or African American Alone','American Indian and Alaska Native Alone','Asian Alone',
             'Native Hawaiian and Other Pacific Islander Alone','Some Other Race Alone','Two or More Races','StateFIPS','CountyFIPS']
data=pd.DataFrame(columns=col_names, data=data.json()[1:])
#let's also add a print line here to let us know when the process is complete
print('Your API call is complete')

Your API call is complete


In [8]:
data.head()

,State,GEOID,Total Population,White Alone,Black or African American Alone,American Indian and Alaska Native Alone,Asian Alone,Native Hawaiian and Other Pacific Islander Alone,Some Other Race Alone,Two or More Races,StateFIPS,CountyFIPS
0,"Baneberry city, Tennessee",1600000US4703078,568,520,0,2,1,8,0,37,47,03078
1,"Lawrenceburg city, Tennessee",1600000US4741340,10877,9782,459,81,214,66,150,125,47,41340
2,"Three Way city, Tennessee",1600000US4774100,1676,1191,429,0,20,0,22,14,47,74100
3,"Millersville city, Tennessee",1600000US4748980,6350,5386,513,0,68,41,236,106,47,48980
4,"Colonial Heights CDP, Tennessee",1600000US4716500,3177,3035,34,13,53,0,10,32,47,16500
